# Faster Hyperparameter Tuning with Scikit-Learn's HalvingGridSearchCV 

If you are a Scikit-Learn fan, Christmas came a few days early in 2020 with the [release of version 0.24.0](https://scikit-learn.org/dev/auto_examples/release_highlights/plot_release_highlights_0_24_0.html#sphx-glr-auto-examples-release-highlights-plot-release-highlights-0-24-0-py). Among the new features are 2 [experimental](https://scikit-learn.org/dev/glossary.html#term-experimental) classes in the model_selection module that support faster hyperparameter optimization: [HalvingGridSearchCV](https://scikit-learn.org/dev/modules/generated/sklearn.model_selection.HalvingGridSearchCV.html) and [HalvingRandomSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.HalvingRandomSearchCV.html#sklearn.model_selection.HalvingRandomSearchCV). 

Like their close cousins [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) and [RandomizedSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html#sklearn.model_selection.RandomizedSearchCV), they use cross-validation to find optimal hyperparameters. However, instead of independently searching the hyperparameter-set candidates, the [successive halving](https://scikit-learn.org/dev/modules/generated/sklearn.model_selection.HalvingGridSearchCV.html) "search strategy starts evaluating all the candidates with a small amount of resources and iteratively selects the best candidates, using more and more resources." The default resource is the number of samples, but the user can set it to any positive-integer model parameter like gradient boosting rounds. Thus, the halving approach has the potential of finding good hyperparameters in less time.


## My Experiment

After reading through Scikit-Learn's "[Comparison between grid search and successive halving](https://scikit-learn.org/dev/auto_examples/model_selection/plot_successive_halving_heatmap.html#sphx-glr-auto-examples-model-selection-plot-successive-halving-heatmap-py)" example (which takes a grand total of 11 seconds to run), I was still unclear about the real-world impact of using the halving approach versus the grid search, so I decided to set up an experiment to answer the following questions:

1. What percentage of time is saved when using HalvingGridSearchCV instead of GridSearchCV?

2. Does HalvingGridSearchCV still select the same hyperparameter set that GridSearchCV does?

I'm going to compare 3 hyperparameter searches: 

1. GridSearchCV

2. HalvingGridSearchCV using the default "n_samples" `resource`
    
3. HalvingGridSearchCV using the CatBoost's "n_estimators" as the `resource`


### Upgrade Scikit-Learn

The first step is to upgrade your version of Scikit to 0.24.0 and make sure you can import the corrent version.



In [1]:
# !! pip install scikit-learn --upgrade
import sklearn
print(sklearn.__version__)

0.24.0


### Loading the Dataset

I ran my tests using the [Kaggle's](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data) Ames, IA house prices dataset. It has 1,460 observatons and 79 features. The dependent variable is the `SalePrice` of the home. I recommend reading [this notebook](https://www.kaggle.com/pmarcelino/comprehensive-data-exploration-with-python) if you are interested in some exploratory data analysis on the dataset.

In [2]:
import numpy as np  
import pandas as pd  

DEP_VAR = 'SalePrice'
train_df = pd.read_csv('../kaggle/input/house-prices-advanced-regression-techniques/train.csv')\
           .set_index('Id')
            
y_train = train_df.pop(DEP_VAR)

### Creating a Pipeline & Model

I also wrote a script called pipeline_ames.py. It instantiates a [Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) containing some feature transformations and the [CatBoostRegressor](https://catboost.ai/docs/concepts/python-reference_catboostregressor.html). I've ploted its visual representation below. (You can read more about my approach to feature engineering in my [previous post](https://towardsdatascience.com/building-columntransformers-dynamically-1-6354bd08aa54).)

In [3]:
from sklearn import set_config                      # to change the display
from sklearn.utils import estimator_html_repr       # to save the diagram into HTML format
from IPython.core.display import display, HTML      # to visualize pipeline

from pipeline_ames import pipe
set_config(display='diagram')
display(HTML(estimator_html_repr(pipe)))

### Experimental Controls 

The `grid_search_params` dictionary contains the control parameters that will be used in each of our trials. We are going to perform 3-fold cross-validation across a `param_grid` containing 4 [CatBoost hyperparameters](https://catboost.ai/docs/concepts/python-reference_parameters-list.html) with 3 values each. The results will be measured in root mean squared log error (RMSLE).

In [4]:
from sklearn.metrics import mean_squared_log_error, make_scorer

np.random.seed(123) # set a global seed
pd.set_option("display.precision", 4)

root_mean_squared_log_error = lambda y_true, y_pred: np.sqrt(mean_squared_log_error(y_true, y_pred))
scorer = make_scorer(root_mean_squared_log_error, greater_is_better=False)

param_grid = {"model__max_depth": [5, 6, 7],
              'model__learning_rate': [.01, 0.03, .06],
              'model__subsample': [.7, .8, .9],
              'model__colsample_bylevel': [.8, .9, 1]}

grid_search_params = dict(estimator=pipe,
                          param_grid=param_grid,
                          scoring=scorer,
                          cv=3,
                          n_jobs=-1,
                          verbose=2)

## Tests

### 1. GridSearchCV

The baseline exhaustive grid search took nearly 30 minutes to perform 3-fold cross-validation on our 81 candidates. We will to see if the HalvingGridSearchCV process can find the same hyperparameters in less time.

In [ ]:
%%time
from sklearn.model_selection import GridSearchCV
full_results = GridSearchCV(**grid_search_params).fit(train_df, y_train)
pd.DataFrame(full_results.best_params_, index=[0]).assign(RMSLE=-full_results.best_score_)

Fitting 3 folds for each of 81 candidates, totalling 243 fits


### 2. HalvingGridSearchCV with n_samples

In first halving grid search, I used the following parameters:

- The default 'n_samples' for the `resource`: This means that the number of training observations will start out small and increase with each iteration

- A `factor` of 2: At the end of an interation, the top half of candidates with be retained and bottom half of candidates will be thrown out. It also means that the next interation will use twice the number of samples. 

- A quarter of the training samples for `min_resources`:  I did not use the default `min_resources` calculation of 22 samples because it produced terrible results. 

Sidenote: If you want the final iteration to use all of the samples, you will need to set  `min_resources` and `factor` to be factors of `max_resources`.

In [ ]:
%%time

from sklearn.experimental import enable_halving_search_cv  
from sklearn.model_selection import HalvingGridSearchCV, GridSearchCV
FACTOR = 2
MAX_RESOURCE_DIVISOR = 4

n_samples = len(train_df)
halving_results_n_samples =\
    HalvingGridSearchCV(resource='n_samples',
                        min_resources=n_samples // MAX_RESOURCE_DIVISOR,
                        factor=FACTOR,
                        **grid_search_params
                        )\
                        .fit(train_df, y_train)

This search did not produce good results. It actually took 5 minutes longer than the exhaustive search. Using my `compare_cv_best_params` function, we see that it found only the ninth optimal hyperparameter set.

In [ ]:
from compare_functions import *

compare_cv_best_params(full_results, *[halving_results_n_samples])\
    .style.apply(lambda row: ['background: red' if row.name == 1 \
                              and col.name == 'full_grid_search_rank' else ''\
                              for col in row], axis=1)

### HalvingGridSearchCV with n_estimators
 
In the second halving search, I used CatBoost's 'n_estimators' as the `resource` and set the first iteration to use a quarter of them. 

In [ ]:
%%time
halving_results_n_estimators =\
    HalvingGridSearchCV(resource='model__n_estimators',                         
                         max_resources=1000,
                         min_resources=1000 // MAX_RESOURCE_DIVISOR,
                         factor=FACTOR,
                        **grid_search_params
                        )\
                        .fit(train_df, y_train)

This halving search produced the results that we were hoping to see. It was about 23% faster than the exhaustive grid search, and it found the best set of hyperparameters.

In [ ]:
compare_cv_best_params(full_results, *[halving_results_n_samples, 
                                       halving_results_n_estimators])\
    .style.apply(lambda row: ['background: lightgreen' if row.name == 2 else '' \
                              for col in row], axis=1)

## Conclusion

The results of my HalvingGridSearchCV experiment were mixed. Using the default "n_samples" `resource` yielded slow and suboptimal results. If you are using an efficient model like CatBoost, limiting the number of samples may not save you any time. However, using CatBoost's `n_estimators` yielded the optimal results in less time. This tracks with my own experience manually tuning CatBoost hyperparameters. I can usually tell pretty quickly from the validation logs whether the hyperparameter set is worth fully exploring.

The original notebook for this blog post can be found [here](https://www.kaggle.com/kylegilde/extracting-scikit-feature-names-importances). Stay tuned for further posts on training & regularizing models with Scikit-Learn. Let me know if you found this post helpful. Thanks!